In [1]:
import main

In [2]:
main.interpret_get_macro("!query_testing " + "~test")

Query<char> query_test;
query_test.OR_queries.push_back(Query<char>::FromString("test"));
pattern.queries.push_back(query_test);


(['Query<char> query_{0};',
  'query_{0}.OR_queries.push_back(Query<char>::FromString("{0}"));',
  'pattern.queries.push_back(query_{});'],
 {'function': 'query_testing', 'args': ['test']})

In [84]:
raw = """//
// Created by Nathaniel Blair on 27/9/20.
//

#include <cassert>

#include "Primal.h"

Primal::Command::Command(vector<string> tags, string command) : tags(tags), command(command) {
// Do nothing
}

Primal::Command::Command(string type, string command) : command(command) {
this->tags = vector<string>();
tags.push_back(type);
}

void Primal::Child(Thread *child) {
this->children.push_back(child);
this->Connect(child);
}

Primal::Primal() :
results(map<int, Prototype>()),
resultsMutex(std::mutex()) {

}

void Primal::NewThread() {
auto* thread = new Thread(this);

Child(thread);
}

void Primal::End() {
for(auto* thread: children) {
thread->endFlag = true;

assert(thread->thread.joinable()); // Checks if the thread is still operating
thread->thread.join();
}
}

void Primal::MergeIntoResult(Prototype *result, unsigned int index) {
std::lock_guard<std::mutex> guard(resultsMutex); // Protects the results-vector from asynchronous competition


if (results.find(index) != results.end())
results[index] = *result;
else
results[index].merge(*result->subvalues_col);
// Tidy up
delete result;
}"""

In [4]:
import re


In [5]:
regex = r'\n\w*[\s::]*\w+::(\w)+\([\w\s<>,*&]*\).*:?w*{'


In [6]:
re.findall(regex, raw)

['d', 'd', 'd', 'd', 'd', 't']

In [90]:
# regex algorithm for unfiltered function names
storage = []
temp_raw = raw

isFailed = False

while not isFailed:
    try:
        f = re.search(r'\n\s*\w*[\s::]*\w+::(\w)+\([\w\:s<>,*&]*\).*:?w*{', temp_raw)

        storage.append(f.group(0))
        temp_raw = temp_raw.replace(f.group(0), "")
        
    except:
        isFailed = True
        
storage

# regex for constructor declaration
regex = r'(\w)*:?:?(\w+)::\2\([\w<>:&*,\s]*\)'

# regex for function name (delimited by '::', just split by '::' and you're good to go!)
regex = r'\w*(::)*\w+::\w+(?=(\([\w<>&*:,\s]*\)))'

# once those constructor names that use : assignments are crossed out, you can use the following to get the remaining function names fully filtered:
regex = r'\w+\([\w<:>,*&\s]*\)'

['\n\nPrimal::Command::Command(vector<string> tags, string command) : tags(tags), command(command) {',
 '\n\nPrimal::Command::Command(string type, string command) : command(command) {',
 '\n\nvoid Primal::Child(Thread *child) {',
 '\n\nvoid Primal::NewThread() {',
 '\n\nvoid Primal::End() {',
 '\n\nvoid Primal::MergeIntoResult(Prototype *result, unsigned int index) {']

In [88]:
# Regex for subclasses/substructs names (unfiltered) 
# NOTE: remember to store if it is a subclass along with it's name
regex = r'::\w+::'

# Once subclasses / substruct results are filtered out:
# Regex for classes / struct names (unfiltered)
# NOTE: remember to store if it is a subclass along with it's name
regex = r'\s\w+::'

In [89]:
# Regex for header files:


# Regex for finding substructs:
regex = r'(struct .+[\s.\w:<>;(),:{*&};]+?)};'
# Regex for finding subclasses:
regex = r'(class .+[\s.\w<:>;(),:{*&};]+?)};'

# Regex for finding classes (with the filename as class name it should be checked for after)
regex = r'(class .+[\s.\w<:>;(),:{*&};]+)};'
# Regex for finding structs (with the filename as struct name it should be checked for after)
regex = r'(struct .+[\s.\w:<>;(),:{*&};]+)};'

In [ ]:
# 0. Create a function called hashmemory, that stores each value given to it into a dict, with a hash and a value as a value and the name as the key, and in a second dict with the hash as key it returns the hash

# 1. If it is a subclass/substruct, replace the class/struct definition with the returned hash; Store every class/struct via hashmemory 
# NOTE: remember to store a reference to the parent class/struct string and whether it is a child struct or not

# 2. For each class/struct in custom_data_structures:

    # 1. Get the string representing a particular struct or class

    # 3. Find all of a certain function within a particular struct or class, with:
    regex = r'(TypeFunctionNameHere\(.*\);)'
    
    # 4. Replace with filtered function names from .cpp files and add a semicolon at the end
    
    # 5. Store the resulting class/struct definition in a dictionary (called 'results') with the name as key
    
# 3. Replace the hashmemory stored sclass/struct declaration (AKA hash) in the parent class/struct with matching (via hashmemory's two dicts) the built string from 'results'

# 4. Replace all the nonsub classes/struct's definitions in the .h file string with the definitions stored by hashmemory in the two dicts
